# 🎯 Transfer Learning YOLO pour Surveillance

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/elfried-kinzoun/intelligent-surveillance-system/blob/main/notebooks/2_transfer_learning_yolo.ipynb)

**Objectif**: Adapter YOLO v8 pour la détection spécialisée en surveillance avec classes personnalisées et optimisations.

## 🎯 Ce que vous allez apprendre :
- 📊 **Préparation dataset** YOLO surveillance
- 🔄 **Transfer learning** depuis COCO vers surveillance
- 🎨 **Classes personnalisées** pour objets de surveillance
- 📈 **Optimisation hyperparamètres** pour votre cas d'usage
- 🚀 **Déploiement** modèle optimisé

## ⚙️ Configuration Recommandée :
- **GPU**: T4 (Colab gratuit) ou A100 (Colab Pro)
- **RAM**: 8+ GB
- **Temps**: 1-2 heures selon dataset

## 🚀 Configuration Initiale

In [ ]:
# Installation des dépendances spécialisées pour YOLO
!pip install -q ultralytics==8.0.0
!pip install -q opencv-python-headless>=4.8.0
!pip install -q pillow>=10.0.0
!pip install -q matplotlib>=3.7.0
!pip install -q seaborn>=0.12.0
!pip install -q wandb  # Weights & Biases pour tracking
!pip install -q albumentations>=1.3.0  # Augmentation de données
!pip install -q roboflow  # Dataset management
!pip install -q supervision  # Visualisation et métriques
!pip install -q torch torchvision --index-url https://download.pytorch.org/whl/cu118

# Installation du projet principal
!git clone -q https://github.com/elfried-kinzoun/intelligent-surveillance-system.git
%cd intelligent-surveillance-system
!pip install -q -e .

print("✅ Installation terminée!")

In [ ]:
import torch
import ultralytics
from ultralytics import YOLO
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import os
import json
from datetime import datetime

# Vérification GPU et configuration
print(f"🔥 PyTorch: {torch.__version__}")
print(f"🎯 Ultralytics: {ultralytics.__version__}")
print(f"🎮 CUDA disponible: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"🎯 GPU: {gpu_name}")
    print(f"💾 Mémoire GPU: {gpu_memory:.1f} GB")
    
    # Configuration optimale selon GPU
    if "T4" in gpu_name:
        TRAINING_CONFIG = {
            "model_size": "yolov8n.pt",  # Nano pour T4
            "batch_size": 16,
            "image_size": 640,
            "epochs": 50,
            "patience": 10
        }
        print("💡 Configuration T4 optimisée")
    elif "A100" in gpu_name or "V100" in gpu_name:
        TRAINING_CONFIG = {
            "model_size": "yolov8m.pt",  # Medium pour GPU puissants
            "batch_size": 32,
            "image_size": 800,
            "epochs": 100,
            "patience": 15
        }
        print("🚀 Configuration haute performance")
    else:
        TRAINING_CONFIG = {
            "model_size": "yolov8s.pt",  # Small par défaut
            "batch_size": 24,
            "image_size": 640,
            "epochs": 75,
            "patience": 12
        }
        print("⚙️ Configuration standard")
else:
    print("⚠️ GPU non disponible - Entraînement CPU (très lent)")
    TRAINING_CONFIG = {
        "model_size": "yolov8n.pt",
        "batch_size": 4,
        "image_size": 416,
        "epochs": 10,
        "patience": 5
    }

print(f"\n⚙️ Configuration sélectionnée:")
for key, value in TRAINING_CONFIG.items():
    print(f"  {key}: {value}")

# Configuration des chemins
DATA_DIR = Path("./data/surveillance_dataset")
MODELS_DIR = Path("./models")
RESULTS_DIR = Path("./results")

# Création des répertoires
DATA_DIR.mkdir(parents=True, exist_ok=True)
MODELS_DIR.mkdir(parents=True, exist_ok=True)
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

print(f"\n📁 Répertoires configurés:")
print(f"  Data: {DATA_DIR}")
print(f"  Models: {MODELS_DIR}")
print(f"  Results: {RESULTS_DIR}")

In [ ]:
# Configuration Weights & Biases pour tracking
import wandb

PROJECT_NAME = "surveillance-yolo-transfer"
RUN_NAME = f"yolo-surveillance-{datetime.now().strftime('%Y%m%d_%H%M')}"

# Initialisation W&B (optionnel)
try:
    # Décommentez si vous avez un compte W&B
    # wandb.login()
    
    wandb.init(
        project=PROJECT_NAME,
        name=RUN_NAME,
        config=TRAINING_CONFIG,
        mode="offline"  # Changez en "online" si compte W&B configuré
    )
    print("✅ W&B configuré")
except Exception as e:
    print(f"⚠️ W&B en mode offline: {e}")
    os.environ["WANDB_MODE"] = "offline"

## 📊 Préparation du Dataset de Surveillance

In [ ]:
# Définition des classes spécialisées pour la surveillance
SURVEILLANCE_CLASSES = {
    # Classes principales COCO adaptées
    0: "person",
    1: "handbag",
    2: "backpack", 
    3: "suitcase",
    4: "bottle",
    5: "cup",
    6: "cell_phone",
    7: "book",
    
    # Classes spécialisées surveillance
    8: "shopping_cart",
    9: "shopping_basket",
    10: "suspicious_object",
    11: "security_camera",
    12: "cash_register",
    13: "product_shelf",
    14: "entrance_exit",
    15: "staff_uniform"
}

# Configuration YAML pour YOLO
dataset_yaml = f"""
# Dataset de surveillance personnalisé
path: {DATA_DIR.absolute()}
train: images/train
val: images/val
test: images/test

# Classes de surveillance
names:
"""

for class_id, class_name in SURVEILLANCE_CLASSES.items():
    dataset_yaml += f"  {class_id}: {class_name}\n"

print("🎯 Classes de surveillance définies:")
for class_id, class_name in SURVEILLANCE_CLASSES.items():
    print(f"  {class_id}: {class_name}")

# Sauvegarde du fichier YAML
yaml_path = DATA_DIR / "dataset.yaml"
with open(yaml_path, "w") as f:
    f.write(dataset_yaml)

print(f"\n✅ Configuration dataset sauvegardée: {yaml_path}")

In [ ]:
import requests
from PIL import Image, ImageDraw, ImageFont
import random

def create_surveillance_dataset(num_images=200):
    """
    Crée un dataset de surveillance synthétique pour la démonstration.
    En production, utilisez vos vraies données annotées.
    """
    
    # URLs d'images de démonstration (stores, shopping, etc.)
    base_images = [
        "https://images.unsplash.com/photo-1556742049-0cfed4f6a45d?w=640",  # Store
        "https://images.unsplash.com/photo-1441986300917-64674bd600d8?w=640",  # Shopping
        "https://images.unsplash.com/photo-1472851294608-062f824d29cc?w=640",  # Retail
        "https://images.unsplash.com/photo-1560472354-b33ff0c44a43?w=640",  # Supermarket
        "https://images.unsplash.com/photo-1578662996442-48f60103fc96?w=640",  # Store aisle
        "https://images.unsplash.com/photo-1607083206869-4c7672e72a8a?w=640",  # Shopping mall
        "https://images.unsplash.com/photo-1596810375465-7846b8b5e0cf?w=640",  # Store interior
        "https://images.unsplash.com/photo-1534723328310-e82dad3ee43f?w=640",  # Grocery
    ]
    
    # Création des dossiers
    for split in ['train', 'val', 'test']:
        (DATA_DIR / 'images' / split).mkdir(parents=True, exist_ok=True)
        (DATA_DIR / 'labels' / split).mkdir(parents=True, exist_ok=True)
    
    # Division du dataset
    train_size = int(0.7 * num_images)
    val_size = int(0.2 * num_images)
    test_size = num_images - train_size - val_size
    
    print(f"📊 Génération dataset ({num_images} images):")
    print(f"  Train: {train_size} | Val: {val_size} | Test: {test_size}")
    
    dataset_info = {"train": [], "val": [], "test": []}
    
    for i in range(num_images):
        # Détermination du split
        if i < train_size:
            split = "train"
        elif i < train_size + val_size:
            split = "val"
        else:
            split = "test"
        
        try:
            # Sélection d'une image de base aléatoire
            base_url = random.choice(base_images)
            
            # Téléchargement (avec cache simple)
            cache_file = DATA_DIR / f"cache_{abs(hash(base_url)) % 1000}.jpg"
            if not cache_file.exists():
                response = requests.get(base_url, timeout=10)
                if response.status_code == 200:
                    with open(cache_file, 'wb') as f:
                        f.write(response.content)
                else:
                    print(f"⚠️ Erreur téléchargement: {base_url}")
                    continue
            
            # Chargement et préparation de l'image
            image = Image.open(cache_file).convert("RGB")
            image = image.resize((TRAINING_CONFIG["image_size"], TRAINING_CONFIG["image_size"]), Image.Resampling.LANCZOS)
            
            # Nom de fichier unique
            filename = f"surveillance_{split}_{i:04d}.jpg"
            
            # Sauvegarde de l'image
            image_path = DATA_DIR / 'images' / split / filename
            image.save(image_path, quality=90)
            
            # Génération d'annotations synthétiques
            annotations = generate_synthetic_annotations(image.size, i)
            
            # Sauvegarde des labels YOLO
            label_path = DATA_DIR / 'labels' / split / filename.replace('.jpg', '.txt')
            with open(label_path, 'w') as f:
                for ann in annotations:
                    f.write(f"{ann['class_id']} {ann['x_center']:.6f} {ann['y_center']:.6f} {ann['width']:.6f} {ann['height']:.6f}\n")
            
            dataset_info[split].append({
                "filename": filename,
                "annotations_count": len(annotations),
                "classes": [ann['class_id'] for ann in annotations]
            })
            
        except Exception as e:
            print(f"❌ Erreur image {i}: {e}")
            continue
        
        if (i + 1) % 20 == 0:
            print(f"  📷 {i + 1}/{num_images} images générées")
    
    return dataset_info

def generate_synthetic_annotations(image_size, seed):
    """
    Génère des annotations synthétiques réalistes.
    """
    random.seed(seed)
    np.random.seed(seed)
    
    annotations = []
    width, height = image_size
    
    # Nombre d'objets aléatoire (1-5)
    num_objects = random.randint(1, 5)
    
    for _ in range(num_objects):
        # Sélection d'une classe selon des probabilités réalistes
        class_probs = {
            0: 0.4,   # person (très commun)
            1: 0.15,  # handbag
            2: 0.1,   # backpack
            4: 0.08,  # bottle
            6: 0.05,  # cell_phone
            8: 0.1,   # shopping_cart
            13: 0.12  # product_shelf
        }
        
        class_id = np.random.choice(
            list(class_probs.keys()), 
            p=list(class_probs.values())
        )
        
        # Génération de bbox réaliste selon la classe
        if class_id == 0:  # person
            w = random.uniform(0.1, 0.3)  # Largeur relative
            h = random.uniform(0.4, 0.8)  # Hauteur relative
        elif class_id in [1, 2]:  # bags
            w = random.uniform(0.05, 0.15)
            h = random.uniform(0.05, 0.15)
        elif class_id == 8:  # shopping_cart
            w = random.uniform(0.15, 0.25)
            h = random.uniform(0.2, 0.35)
        elif class_id == 13:  # product_shelf
            w = random.uniform(0.3, 0.8)
            h = random.uniform(0.2, 0.6)
        else:
            w = random.uniform(0.03, 0.12)
            h = random.uniform(0.03, 0.12)
        
        # Position aléatoire en évitant les bords
        x_center = random.uniform(w/2 + 0.05, 1 - w/2 - 0.05)
        y_center = random.uniform(h/2 + 0.05, 1 - h/2 - 0.05)
        
        annotations.append({
            "class_id": class_id,
            "x_center": x_center,
            "y_center": y_center,
            "width": w,
            "height": h
        })
    
    return annotations

# Génération du dataset
print("🏗️ Création du dataset de surveillance...")
dataset_info = create_surveillance_dataset(num_images=100)  # Réduire pour démo rapide

print("\n📊 Statistiques du dataset:")
for split, info in dataset_info.items():
    total_annotations = sum(item["annotations_count"] for item in info)
    print(f"  {split.upper()}: {len(info)} images, {total_annotations} annotations")

print("\n✅ Dataset créé avec succès!")

In [ ]:
import supervision as sv
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def visualize_dataset_samples(num_samples=6):
    """
    Visualise des échantillons du dataset avec annotations.
    """
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.flatten()
    
    # Sélection d'échantillons du train set
    train_images = list((DATA_DIR / 'images' / 'train').glob('*.jpg'))
    selected_images = random.sample(train_images, min(num_samples, len(train_images)))
    
    for i, image_path in enumerate(selected_images):
        # Chargement de l'image
        image = cv2.imread(str(image_path))
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Chargement des annotations
        label_path = DATA_DIR / 'labels' / 'train' / (image_path.stem + '.txt')
        
        ax = axes[i]
        ax.imshow(image_rgb)
        
        if label_path.exists():
            with open(label_path, 'r') as f:
                annotations = f.readlines()
            
            h, w = image_rgb.shape[:2]
            
            for ann in annotations:
                parts = ann.strip().split()
                if len(parts) == 5:
                    class_id, x_center, y_center, width, height = map(float, parts)
                    class_id = int(class_id)
                    
                    # Conversion vers coordonnées absolues
                    x1 = int((x_center - width/2) * w)
                    y1 = int((y_center - height/2) * h)
                    x2 = int((x_center + width/2) * w)
                    y2 = int((y_center + height/2) * h)
                    
                    # Dessin du rectangle
                    rect = patches.Rectangle(
                        (x1, y1), x2-x1, y2-y1,
                        linewidth=2, 
                        edgecolor=plt.cm.Set3(class_id / len(SURVEILLANCE_CLASSES)),
                        facecolor='none'
                    )
                    ax.add_patch(rect)
                    
                    # Label de classe
                    class_name = SURVEILLANCE_CLASSES.get(class_id, f"class_{class_id}")
                    ax.text(
                        x1, y1-5, class_name,
                        color=plt.cm.Set3(class_id / len(SURVEILLANCE_CLASSES)),
                        fontsize=8, fontweight='bold',
                        bbox=dict(boxstyle="round,pad=0.3", facecolor='white', alpha=0.7)
                    )
        
        ax.set_title(f"Sample {i+1}: {image_path.name}")
        ax.axis('off')
    
    plt.tight_layout()
    plt.suptitle("📊 Échantillons du Dataset de Surveillance", fontsize=16, y=1.02)
    plt.show()

# Analyse des classes
def analyze_dataset_distribution():
    """
    Analyse la distribution des classes dans le dataset.
    """
    
    class_counts = {class_id: 0 for class_id in SURVEILLANCE_CLASSES.keys()}
    
    # Parcours de tous les fichiers de labels
    for split in ['train', 'val', 'test']:
        labels_dir = DATA_DIR / 'labels' / split
        if labels_dir.exists():
            for label_file in labels_dir.glob('*.txt'):
                with open(label_file, 'r') as f:
                    for line in f:
                        parts = line.strip().split()
                        if len(parts) >= 1:
                            class_id = int(float(parts[0]))
                            if class_id in class_counts:
                                class_counts[class_id] += 1
    
    # Visualisation de la distribution
    plt.figure(figsize=(14, 6))
    
    class_names = [SURVEILLANCE_CLASSES[cid] for cid in sorted(class_counts.keys())]
    counts = [class_counts[cid] for cid in sorted(class_counts.keys())]
    
    bars = plt.bar(class_names, counts, alpha=0.8)
    plt.title('📊 Distribution des Classes dans le Dataset')
    plt.xlabel('Classes')
    plt.ylabel('Nombre d\'Annotations')
    plt.xticks(rotation=45, ha='right')
    
    # Ajout des valeurs sur les barres
    for bar, count in zip(bars, counts):
        if count > 0:
            plt.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.5,
                    f'{count}', ha='center', va='bottom', fontsize=9)
    
    plt.tight_layout()
    plt.show()
    
    return class_counts

# Visualisation des échantillons
print("🖼️ Visualisation des échantillons du dataset...")
visualize_dataset_samples()

# Analyse de la distribution
print("\n📈 Analyse de la distribution des classes...")
class_distribution = analyze_dataset_distribution()

print("\n📊 Résumé de la distribution:")
total_annotations = sum(class_distribution.values())
for class_id, count in class_distribution.items():
    if count > 0:
        percentage = (count / total_annotations) * 100
        print(f"  {SURVEILLANCE_CLASSES[class_id]}: {count} ({percentage:.1f}%)")

print(f"\n✅ Total annotations: {total_annotations}")

## 🧠 Configuration du Modèle YOLO

In [ ]:
from ultralytics import YOLO
import torch

# Chargement du modèle pré-entraîné
model_size = TRAINING_CONFIG["model_size"]
print(f"🧠 Chargement du modèle YOLO: {model_size}")

# Téléchargement et chargement du modèle
model = YOLO(model_size)

print(f"✅ Modèle {model_size} chargé")
print(f"📊 Nombre de classes COCO originales: {len(model.names)}")
print(f"📊 Nouvelles classes surveillance: {len(SURVEILLANCE_CLASSES)}")

# Information sur le modèle
model_info = model.info(verbose=False)
print(f"\n🔧 Architecture du modèle:")
print(f"  Paramètres: ~{sum(p.numel() for p in model.model.parameters())/1e6:.1f}M")
print(f"  GFLOPS: {model.model.model[-1].anchors.numel() if hasattr(model.model.model[-1], 'anchors') else 'N/A'}")
print(f"  Device: {next(model.model.parameters()).device}")

# Vérification GPU
if torch.cuda.is_available():
    print(f"🎮 Modèle sur GPU: {torch.cuda.get_device_name()}")
    print(f"💾 Mémoire GPU utilisée: {torch.cuda.memory_allocated()/1e9:.1f} GB")
else:
    print("⚠️ Modèle sur CPU")

# Test de prédiction pour vérifier le fonctionnement
print("\n🧪 Test de prédiction...")
test_image_path = list((DATA_DIR / 'images' / 'train').glob('*.jpg'))[0]
test_results = model.predict(test_image_path, verbose=False)
print(f"✅ Test réussi - {len(test_results[0].boxes)} détections initiales")

In [ ]:
# Configuration avancée pour l'entraînement
training_args = {
    # Dataset
    "data": str(yaml_path),
    
    # Hyperparamètres de base
    "epochs": TRAINING_CONFIG["epochs"],
    "batch": TRAINING_CONFIG["batch_size"],
    "imgsz": TRAINING_CONFIG["image_size"],
    
    # Optimiseur
    "optimizer": "AdamW",  # AdamW généralement meilleur que SGD
    "lr0": 0.001,  # Learning rate initial
    "lrf": 0.01,   # Learning rate final (lr0 * lrf)
    "momentum": 0.937,
    "weight_decay": 0.0005,
    
    # Scheduler
    "warmup_epochs": 3,
    "warmup_momentum": 0.8,
    "warmup_bias_lr": 0.1,
    
    # Augmentation de données
    "hsv_h": 0.015,      # Hue augmentation
    "hsv_s": 0.7,        # Saturation augmentation  
    "hsv_v": 0.4,        # Value augmentation
    "degrees": 0.0,      # Rotation (disabled pour surveillance)
    "translate": 0.1,    # Translation
    "scale": 0.5,        # Scale augmentation
    "shear": 0.0,        # Shear (disabled)
    "perspective": 0.0,  # Perspective (disabled)
    "flipud": 0.0,       # Flip up-down (disabled)
    "fliplr": 0.5,       # Flip left-right
    "mosaic": 1.0,       # Mosaic augmentation
    "mixup": 0.0,        # Mixup augmentation (disabled)
    "copy_paste": 0.0,   # Copy-paste augmentation (disabled)
    
    # Loss function
    "box": 7.5,          # Box loss gain
    "cls": 0.5,          # Class loss gain  
    "dfl": 1.5,          # DFL loss gain
    
    # Validation et sauvegarde
    "patience": TRAINING_CONFIG["patience"],
    "save": True,
    "save_period": 10,   # Sauvegarder chaque 10 époques
    "cache": "ram",      # Cache en RAM pour accélérer
    
    # Monitoring
    "project": str(RESULTS_DIR),
    "name": f"surveillance_yolo_{datetime.now().strftime('%Y%m%d_%H%M')}",
    "verbose": True,
    "plots": True,       # Générer les plots automatiquement
    
    # Performance
    "workers": 8 if torch.cuda.is_available() else 2,
    "device": 0 if torch.cuda.is_available() else "cpu",
}

print("⚙️ Configuration d'entraînement:")
for key, value in training_args.items():
    print(f"  {key}: {value}")

# Estimation du temps d'entraînement
dataset_size = sum(len(info) for info in dataset_info.values())
estimated_time_per_epoch = (dataset_size / TRAINING_CONFIG["batch_size"]) * 0.5  # 0.5s par batch estimé
total_estimated_time = estimated_time_per_epoch * TRAINING_CONFIG["epochs"] / 60

print(f"\n⏱️ Estimation temps d'entraînement:")
print(f"  ~{estimated_time_per_epoch:.1f}s par époque")
print(f"  ~{total_estimated_time:.1f} minutes au total")
print(f"  Dataset: {dataset_size} images")
print(f"  Batch size: {TRAINING_CONFIG['batch_size']}")

# Vérification de la mémoire GPU si disponible
if torch.cuda.is_available():
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    allocated_memory = torch.cuda.memory_allocated() / 1e9
    print(f"\n💾 Mémoire GPU: {allocated_memory:.1f}/{gpu_memory:.1f} GB")
    
    if allocated_memory / gpu_memory > 0.8:
        print("⚠️ Mémoire GPU élevée - Réduisez batch_size si nécessaire")
    else:
        print("✅ Mémoire GPU OK pour l'entraînement")

## 🚀 Lancement de l'Entraînement

In [ ]:
import time
from datetime import datetime

print("🚀 DÉBUT DE L'ENTRAÎNEMENT YOLO")
print("=" * 50)
print(f"🕐 Heure de début: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"📊 Modèle: {model_size}")
print(f"📈 Époques: {TRAINING_CONFIG['epochs']}")
print(f"📦 Batch size: {TRAINING_CONFIG['batch_size']}")
print(f"🖼️ Image size: {TRAINING_CONFIG['image_size']}")
print(f"💾 Dataset: {sum(len(info) for info in dataset_info.values())} images")
print()

# Sauvegarde des métriques initiales
start_time = time.time()
initial_gpu_memory = torch.cuda.memory_allocated() / 1e9 if torch.cuda.is_available() else 0

try:
    # Lancement de l'entraînement
    print("📚 Démarrage de l'entraînement...")
    print("📊 Métriques à surveiller: mAP50, mAP50-95, Precision, Recall")
    print("⚠️  L'entraînement peut prendre du temps selon votre GPU...")
    print()
    
    # Entraînement du modèle
    results = model.train(**training_args)
    
    # Calcul du temps d'entraînement
    end_time = time.time()
    training_duration = end_time - start_time
    final_gpu_memory = torch.cuda.memory_allocated() / 1e9 if torch.cuda.is_available() else 0
    
    print("\n" + "=" * 50)
    print("🎉 ENTRAÎNEMENT TERMINÉ AVEC SUCCÈS !")
    print("=" * 50)
    print(f"🕐 Durée totale: {training_duration/60:.1f} minutes")
    print(f"⚡ Temps par époque: {training_duration/TRAINING_CONFIG['epochs']:.1f}s")
    print(f"💾 Mémoire GPU finale: {final_gpu_memory:.1f} GB")
    
    # Extraction des métriques finales
    if hasattr(results, 'results_dict'):
        metrics = results.results_dict
        print(f"\n📊 Métriques finales:")
        if 'metrics/mAP50(B)' in metrics:
            print(f"  mAP@0.5: {metrics['metrics/mAP50(B)']:.3f}")
        if 'metrics/mAP50-95(B)' in metrics:
            print(f"  mAP@0.5-0.95: {metrics['metrics/mAP50-95(B)']:.3f}")
        if 'metrics/precision(B)' in metrics:
            print(f"  Précision: {metrics['metrics/precision(B)']:.3f}")
        if 'metrics/recall(B)' in metrics:
            print(f"  Rappel: {metrics['metrics/recall(B)']:.3f}")
    
    # Sauvegarde du modèle final
    model_save_path = MODELS_DIR / f"surveillance_yolo_{datetime.now().strftime('%Y%m%d_%H%M')}.pt"
    model.save(model_save_path)
    print(f"\n💾 Modèle sauvegardé: {model_save_path}")
    
    # Log vers W&B si configuré
    if 'wandb' in globals() and wandb.run:
        wandb.log({
            "training_duration_minutes": training_duration/60,
            "final_gpu_memory_gb": final_gpu_memory,
            "model_size": model_size,
            "dataset_size": sum(len(info) for info in dataset_info.values())
        })
    
    training_success = True
    
except Exception as e:
    print(f"\n❌ ERREUR PENDANT L'ENTRAÎNEMENT: {e}")
    print("\n💡 Solutions possibles:")
    print("  - Réduire le batch_size")
    print("  - Réduire la taille d'image (imgsz)")
    print("  - Utiliser un modèle plus petit (yolov8n.pt)")
    print("  - Vérifier le format du dataset")
    print("  - Libérer la mémoire GPU")
    
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print("🧹 Cache GPU vidé")
    
    training_success = False
    raise e

## 📊 Évaluation du Modèle

In [ ]:
# Évaluation complète du modèle entraîné
print("📊 ÉVALUATION DU MODÈLE ENTRAÎNÉ")
print("=" * 40)

if training_success:
    # Évaluation sur le set de validation
    print("📈 Évaluation sur le set de validation...")
    
    try:
        # Validation metrics
        val_results = model.val(data=str(yaml_path), split='val')
        
        print("\n📊 Métriques de validation:")
        if hasattr(val_results, 'results_dict'):
            metrics = val_results.results_dict
            for key, value in metrics.items():
                if isinstance(value, (int, float)):
                    print(f"  {key}: {value:.4f}")
        
        # Métriques par classe si disponibles
        if hasattr(val_results, 'ap_class_index'):
            print("\n📋 Métriques par classe:")
            for i, class_idx in enumerate(val_results.ap_class_index):
                if class_idx in SURVEILLANCE_CLASSES:
                    class_name = SURVEILLANCE_CLASSES[class_idx]
                    ap50 = val_results.ap50[i] if hasattr(val_results, 'ap50') else 0
                    print(f"  {class_name}: AP@0.5 = {ap50:.3f}")
        
    except Exception as e:
        print(f"⚠️ Erreur évaluation: {e}")
    
    # Test sur quelques images
    print("\n🧪 Test sur images de validation...")
    
    val_images = list((DATA_DIR / 'images' / 'val').glob('*.jpg'))[:6]
    
    if val_images:
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        axes = axes.flatten()
        
        for i, image_path in enumerate(val_images):
            # Prédiction
            results = model.predict(image_path, conf=0.25, verbose=False)
            
            # Chargement de l'image
            image = cv2.imread(str(image_path))
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            ax = axes[i]
            ax.imshow(image_rgb)
            
            # Affichage des prédictions
            if len(results[0].boxes) > 0:
                boxes = results[0].boxes
                for j in range(len(boxes)):
                    # Coordonnées de la boîte
                    x1, y1, x2, y2 = boxes.xyxy[j].cpu().numpy()
                    conf = boxes.conf[j].cpu().numpy()
                    cls = int(boxes.cls[j].cpu().numpy())
                    
                    # Dessin du rectangle
                    rect = patches.Rectangle(
                        (x1, y1), x2-x1, y2-y1,
                        linewidth=2,
                        edgecolor='red',
                        facecolor='none'
                    )
                    ax.add_patch(rect)
                    
                    # Label avec confiance
                    class_name = SURVEILLANCE_CLASSES.get(cls, f"class_{cls}")
                    ax.text(
                        x1, y1-5, f"{class_name}: {conf:.2f}",
                        color='red', fontsize=8, fontweight='bold',
                        bbox=dict(boxstyle="round,pad=0.3", facecolor='yellow', alpha=0.7)
                    )
            
            ax.set_title(f"Prédictions: {image_path.name}")
            ax.axis('off')
        
        plt.tight_layout()
        plt.suptitle("🎯 Prédictions du Modèle sur Set de Validation", fontsize=16, y=1.02)
        plt.show()
    
else:
    print("⚠️ Pas d'évaluation possible - Entraînement échoué")

print("\n✅ Évaluation terminée")

In [ ]:
# Comparaison avec le modèle pré-entraîné COCO
print("🆚 COMPARAISON MODÈLE COCO vs SURVEILLANCE")
print("=" * 50)

if training_success:
    # Chargement du modèle COCO original pour comparaison
    coco_model = YOLO(model_size)
    
    # Test sur une image de validation
    test_image = list((DATA_DIR / 'images' / 'val').glob('*.jpg'))[0]
    
    print(f"🖼️ Test comparatif sur: {test_image.name}")
    
    # Prédictions modèle COCO
    coco_results = coco_model.predict(test_image, conf=0.25, verbose=False)
    
    # Prédictions modèle surveillance
    surveillance_results = model.predict(test_image, conf=0.25, verbose=False)
    
    # Comparaison visuelle
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
    
    # Chargement de l'image
    image = cv2.imread(str(test_image))
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Modèle COCO
    ax1.imshow(image_rgb)
    if len(coco_results[0].boxes) > 0:
        boxes = coco_results[0].boxes
        for j in range(len(boxes)):
            x1, y1, x2, y2 = boxes.xyxy[j].cpu().numpy()
            conf = boxes.conf[j].cpu().numpy()
            cls = int(boxes.cls[j].cpu().numpy())
            
            rect = patches.Rectangle(
                (x1, y1), x2-x1, y2-y1,
                linewidth=2, edgecolor='blue', facecolor='none'
            )
            ax1.add_patch(rect)
            
            # Nom de classe COCO
            class_name = coco_model.names[cls]
            ax1.text(
                x1, y1-5, f"{class_name}: {conf:.2f}",
                color='blue', fontsize=8, fontweight='bold',
                bbox=dict(boxstyle="round,pad=0.3", facecolor='lightblue', alpha=0.7)
            )
    
    ax1.set_title(f"Modèle COCO Original\n{len(coco_results[0].boxes)} détections")
    ax1.axis('off')
    
    # Modèle surveillance
    ax2.imshow(image_rgb)
    if len(surveillance_results[0].boxes) > 0:
        boxes = surveillance_results[0].boxes
        for j in range(len(boxes)):
            x1, y1, x2, y2 = boxes.xyxy[j].cpu().numpy()
            conf = boxes.conf[j].cpu().numpy()
            cls = int(boxes.cls[j].cpu().numpy())
            
            rect = patches.Rectangle(
                (x1, y1), x2-x1, y2-y1,
                linewidth=2, edgecolor='red', facecolor='none'
            )
            ax2.add_patch(rect)
            
            # Nom de classe surveillance
            class_name = SURVEILLANCE_CLASSES.get(cls, f"class_{cls}")
            ax2.text(
                x1, y1-5, f"{class_name}: {conf:.2f}",
                color='red', fontsize=8, fontweight='bold',
                bbox=dict(boxstyle="round,pad=0.3", facecolor='yellow', alpha=0.7)
            )
    
    ax2.set_title(f"Modèle Surveillance Fine-tuné\n{len(surveillance_results[0].boxes)} détections")
    ax2.axis('off')
    
    plt.tight_layout()
    plt.suptitle("🔍 Comparaison Modèles COCO vs Surveillance", fontsize=16, y=0.98)
    plt.show()
    
    # Analyse comparative
    print("\n📊 Analyse comparative:")
    print(f"  Détections COCO: {len(coco_results[0].boxes)}")
    print(f"  Détections Surveillance: {len(surveillance_results[0].boxes)}")
    
    # Classes détectées
    if len(coco_results[0].boxes) > 0:
        coco_classes = [coco_model.names[int(cls)] for cls in coco_results[0].boxes.cls.cpu().numpy()]
        print(f"  Classes COCO: {set(coco_classes)}")
    
    if len(surveillance_results[0].boxes) > 0:
        surv_classes = [SURVEILLANCE_CLASSES.get(int(cls), f"class_{cls}") for cls in surveillance_results[0].boxes.cls.cpu().numpy()]
        print(f"  Classes Surveillance: {set(surv_classes)}")
    
    print("\n💡 Observations attendues:")
    print("  • Le modèle surveillance devrait mieux détecter les objets spécialisés")
    print("  • Les classes de surveillance sont plus spécifiques au contexte")
    print("  • La précision devrait être améliorée pour les scénarios ciblés")

else:
    print("⚠️ Comparaison impossible - Entraînement non réussi")

print("\n✅ Comparaison terminée")

## ⚡ Optimisation et Export

In [ ]:
# Export du modèle dans différents formats pour déploiement
print("⚡ EXPORT ET OPTIMISATION DU MODÈLE")
print("=" * 45)

if training_success:
    export_formats = {
        "ONNX": "onnx",        # Standard inter-frameworks
        "TensorRT": "engine",  # NVIDIA optimisé (si GPU NVIDIA)
        "TorchScript": "torchscript", # PyTorch optimisé
        "OpenVINO": "openvino",     # Intel optimisé
    }
    
    exported_models = {}
    
    for format_name, format_ext in export_formats.items():
        try:
            print(f"\n📦 Export {format_name}...")
            
            if format_name == "TensorRT" and not torch.cuda.is_available():
                print(f"  ⏭️ TensorRT ignoré (pas de GPU NVIDIA)")
                continue
                
            if format_name == "OpenVINO":
                print(f"  ⏭️ OpenVINO ignoré (installation complexe sur Colab)")
                continue
            
            # Export du modèle
            export_path = model.export(
                format=format_ext,
                imgsz=TRAINING_CONFIG["image_size"],
                optimize=True,
                half=torch.cuda.is_available(),  # FP16 si GPU
                device=0 if torch.cuda.is_available() else "cpu"
            )
            
            exported_models[format_name] = export_path
            print(f"  ✅ {format_name} exporté: {export_path}")
            
        except Exception as e:
            print(f"  ❌ Erreur export {format_name}: {e}")
    
    # Test des performances des modèles exportés
    print("\n⚡ Benchmark des performances:")
    
    test_image = list((DATA_DIR / 'images' / 'val').glob('*.jpg'))[0]
    
    # Test modèle PyTorch original
    start_time = time.time()
    for _ in range(10):
        _ = model.predict(test_image, verbose=False)
    pytorch_time = (time.time() - start_time) / 10
    print(f"  PyTorch: {pytorch_time*1000:.1f}ms/image")
    
    # Test modèles exportés
    for format_name, model_path in exported_models.items():
        try:
            # Chargement du modèle exporté
            exported_model = YOLO(model_path)
            
            start_time = time.time()
            for _ in range(10):
                _ = exported_model.predict(test_image, verbose=False)
            export_time = (time.time() - start_time) / 10
            
            speedup = pytorch_time / export_time
            print(f"  {format_name}: {export_time*1000:.1f}ms/image (speedup: {speedup:.1f}x)")
            
        except Exception as e:
            print(f"  {format_name}: Erreur benchmark - {e}")

else:
    print("⚠️ Export impossible - Modèle non entraîné")

print("\n✅ Optimisation terminée")

In [ ]:
# Génération du code d'intégration pour le système principal
print("🔗 GÉNÉRATION CODE D'INTÉGRATION")
print("=" * 40)

if training_success:
    # Code d'intégration personnalisé
    integration_template = f'''
# Intégration du modèle YOLO fine-tuné dans le système de surveillance
# Remplacez dans src/detection/yolo/detector.py

from ultralytics import YOLO
import torch
from pathlib import Path

class SurveillanceYOLODetector:
    """
    Détecteur YOLO spécialisé pour la surveillance.
    """
    
    # Classes spécialisées surveillance
    SURVEILLANCE_CLASSES = {{
        {', '.join([f'{k}: "{v}"' for k, v in SURVEILLANCE_CLASSES.items()])}
    }}
    
    def __init__(self, 
                 model_path: str = "models/surveillance_yolo.pt",
                 confidence_threshold: float = 0.25,
                 device: str = "auto"):
        
        self.model_path = model_path
        self.confidence_threshold = confidence_threshold
        self.device = self._setup_device(device)
        
        # Chargement du modèle fine-tuné
        if Path(model_path).exists():
            self.model = YOLO(model_path)
            print(f"✅ Modèle surveillance chargé: {{model_path}}")
        else:
            print(f"⚠️ Modèle introuvable: {{model_path}} - Utilisation modèle standard")
            self.model = YOLO("yolov8n.pt")
        
        self.is_loaded = True
    
    def detect(self, frame, custom_confidence=None):
        """Détection avec le modèle fine-tuné."""
        
        conf = custom_confidence or self.confidence_threshold
        
        # Prédiction
        results = self.model.predict(
            frame.image,
            conf=conf,
            verbose=False,
            device=self.device
        )
        
        # Conversion vers le format DetectedObject
        detections = []
        
        if len(results[0].boxes) > 0:
            boxes = results[0].boxes
            for i in range(len(boxes)):
                x1, y1, x2, y2 = boxes.xyxy[i].cpu().numpy()
                conf = float(boxes.conf[i].cpu().numpy())
                cls = int(boxes.cls[i].cpu().numpy())
                
                # Création BoundingBox
                bbox = BoundingBox(
                    x=int(x1),
                    y=int(y1), 
                    width=int(x2-x1),
                    height=int(y2-y1),
                    confidence=conf
                )
                
                # Création DetectedObject
                detection = DetectedObject(
                    class_id=cls,
                    class_name=self.SURVEILLANCE_CLASSES.get(cls, f"class_{{cls}}"),
                    bbox=bbox,
                    confidence=conf
                )
                
                detections.append(detection)
        
        return detections
    
    def _setup_device(self, device):
        if device == "auto":
            return "cuda" if torch.cuda.is_available() else "cpu"
        return device

# Usage dans le système principal:
# detector = SurveillanceYOLODetector(
#     model_path="models/surveillance_yolo_optimized.pt",
#     confidence_threshold=0.3
# )
'''
    
    # Sauvegarde du code d'intégration
    integration_path = RESULTS_DIR / "surveillance_yolo_integration.py"
    with open(integration_path, 'w') as f:
        f.write(integration_template)
    
    print(f"✅ Code d'intégration généré: {integration_path}")
    
    # Instructions d'utilisation
    print("\n📋 INSTRUCTIONS D'INTÉGRATION:")
    print("1. 📁 Copiez le modèle .pt dans le dossier models/ du projet")
    print("2. 🔧 Remplacez YOLODetector par SurveillanceYOLODetector")
    print("3. 🧪 Testez avec vos données de surveillance réelles")
    print("4. ⚙️ Ajustez confidence_threshold selon vos besoins")
    print("5. 📊 Surveillez les métriques de performance en production")
    
    # Configuration recommandée
    print("\n⚙️ CONFIGURATION RECOMMANDÉE:")
    print(f"  • Seuil confiance: 0.25-0.4 (selon précision/rappel souhaités)")
    print(f"  • Taille image: {TRAINING_CONFIG['image_size']} (même qu'entraînement)")
    print(f"  • Device: {'CUDA' if torch.cuda.is_available() else 'CPU'}")
    print(f"  • Format optimal: {'ONNX ou TensorRT' if torch.cuda.is_available() else 'ONNX'}")
    
else:
    print("⚠️ Génération impossible - Modèle non entraîné")

print("\n✅ Intégration préparée")

## 💾 Sauvegarde et Déploiement

In [ ]:
# Sauvegarde complète des artefacts d'entraînement
print("💾 SAUVEGARDE DES ARTEFACTS")
print("=" * 35)

if training_success:
    # Création du dossier de sauvegarde
    save_dir = RESULTS_DIR / f"surveillance_yolo_complete_{datetime.now().strftime('%Y%m%d_%H%M')}"
    save_dir.mkdir(exist_ok=True)
    
    # 1. Modèle principal
    print("📦 Sauvegarde modèle et poids...")
    model_save_path = save_dir / "surveillance_yolo.pt"
    model.save(model_save_path)
    print(f"  ✅ Modèle: {model_save_path}")
    
    # 2. Configuration dataset
    import shutil
    dataset_config_path = save_dir / "dataset.yaml"
    shutil.copy(yaml_path, dataset_config_path)
    print(f"  ✅ Config dataset: {dataset_config_path}")
    
    # 3. Métriques et logs d'entraînement
    training_metadata = {
        "model_info": {
            "base_model": model_size,
            "classes_count": len(SURVEILLANCE_CLASSES),
            "classes": SURVEILLANCE_CLASSES,
        },
        "training_config": TRAINING_CONFIG,
        "training_args": {k: str(v) for k, v in training_args.items()},
        "dataset_info": {
            "total_images": sum(len(info) for info in dataset_info.values()),
            "splits": {k: len(v) for k, v in dataset_info.items()},
            "class_distribution": class_distribution
        },
        "performance": {
            "training_duration_minutes": training_duration/60 if 'training_duration' in locals() else None,
            "gpu_used": torch.cuda.get_device_name() if torch.cuda.is_available() else "CPU",
            "final_metrics": getattr(results, 'results_dict', {}) if 'results' in locals() else {}
        },
        "export_info": {
            "exported_formats": list(exported_models.keys()) if 'exported_models' in locals() else [],
            "pytorch_inference_ms": pytorch_time*1000 if 'pytorch_time' in locals() else None
        },
        "created_at": datetime.now().isoformat()
    }
    
    metadata_path = save_dir / "training_metadata.json"
    with open(metadata_path, 'w') as f:
        json.dump(training_metadata, f, indent=2, default=str)
    print(f"  ✅ Métadonnées: {metadata_path}")
    
    # 4. Code d'intégration
    integration_dest = save_dir / "integration_code.py"
    if integration_path.exists():
        shutil.copy(integration_path, integration_dest)
        print(f"  ✅ Code intégration: {integration_dest}")
    
    # 5. Modèles exportés
    if 'exported_models' in locals():
        exports_dir = save_dir / "exported_models"
        exports_dir.mkdir(exist_ok=True)
        
        for format_name, model_path in exported_models.items():
            try:
                dest_path = exports_dir / f"surveillance_yolo.{format_name.lower()}"
                shutil.copy(model_path, dest_path)
                print(f"  ✅ Export {format_name}: {dest_path}")
            except Exception as e:
                print(f"  ⚠️ Erreur copie {format_name}: {e}")
    
    # 6. Documentation README
    readme_content = f'''
# 🎯 Modèle YOLO Surveillance Fine-tuné

## Description
Modèle YOLO v8 spécialement fine-tuné pour la détection d'objets en contexte de surveillance de grande distribution.

## Informations Modèle
- **Modèle de base**: {model_size}
- **Classes**: {len(SURVEILLANCE_CLASSES)} classes spécialisées
- **Entraînement**: {TRAINING_CONFIG["epochs"]} époques
- **Dataset**: {sum(len(info) for info in dataset_info.values())} images
- **GPU utilisé**: {torch.cuda.get_device_name() if torch.cuda.is_available() else "CPU"}
- **Date création**: {datetime.now().strftime('%Y-%m-%d %H:%M')}

## Classes Détectées
```python
CLASSES = {{
    {chr(10).join([f"    {k}: '{v}'" for k, v in SURVEILLANCE_CLASSES.items()])}
}}
```

## Utilisation
```python
from ultralytics import YOLO

# Chargement du modèle
model = YOLO('surveillance_yolo.pt')

# Prédiction
results = model.predict('image.jpg', conf=0.25)

# Affichage des résultats
results[0].show()
```

## Performance
- **Précision optimisée** pour contexte surveillance
- **Classes spécialisées** adaptées aux magasins
- **Transfer learning** depuis COCO pour rapidité
{f"- **Temps inférence**: ~{pytorch_time*1000:.0f}ms par image" if 'pytorch_time' in locals() else ""}

## Intégration
Consultez `integration_code.py` pour l'intégration dans le système de surveillance complet.

## Fichiers Inclus
- `surveillance_yolo.pt` - Modèle principal
- `dataset.yaml` - Configuration classes
- `training_metadata.json` - Métriques d'entraînement
- `integration_code.py` - Code d'intégration
- `exported_models/` - Modèles optimisés (ONNX, etc.)

## Support
Pour questions ou améliorations, consultez la documentation du système de surveillance complet.
'''
    
    readme_path = save_dir / "README.md"
    with open(readme_path, 'w') as f:
        f.write(readme_content)
    print(f"  ✅ Documentation: {readme_path}")
    
    # 7. Archive complète
    print("\n📦 Création de l'archive...")
    archive_name = f"surveillance_yolo_{datetime.now().strftime('%Y%m%d_%H%M')}"
    
    try:
        archive_path = shutil.make_archive(
            str(RESULTS_DIR / archive_name), 
            'zip', 
            str(save_dir)
        )
        print(f"  ✅ Archive créée: {archive_path}")
        
        # Taille de l'archive
        archive_size = Path(archive_path).stat().st_size / (1024*1024)
        print(f"  📊 Taille archive: {archive_size:.1f} MB")
        
    except Exception as e:
        print(f"  ⚠️ Erreur création archive: {e}")
    
    # 8. Téléchargement sur Colab
    if 'google.colab' in str(get_ipython()):
        try:
            from google.colab import files
            print("\n📥 Téléchargement sur Colab...")
            
            # Télécharger l'archive
            if 'archive_path' in locals() and Path(archive_path).exists():
                files.download(archive_path)
                print("  ✅ Archive téléchargée")
            
            # Télécharger le modèle principal
            files.download(str(model_save_path))
            print("  ✅ Modèle téléchargé")
            
        except Exception as e:
            print(f"  ⚠️ Téléchargement impossible: {e}")
    
    print(f"\n✅ Sauvegarde terminée dans: {save_dir}")
    
else:
    print("⚠️ Pas de sauvegarde - Entraînement non réussi")

print("\n📋 FICHIERS GÉNÉRÉS:")
if training_success:
    print(f"  📁 Dossier principal: {save_dir}")
    print(f"  🤖 Modèle YOLO: surveillance_yolo.pt")
    print(f"  ⚙️ Configuration: dataset.yaml")
    print(f"  📊 Métadonnées: training_metadata.json")
    print(f"  🔧 Code intégration: integration_code.py")
    print(f"  📖 Documentation: README.md")
    print(f"  📦 Archive: {archive_name}.zip (si créée)")
print("\n💾 Sauvegarde terminée !")

## 🎯 Conclusion et Prochaines Étapes

### ✅ Ce que vous avez accompli :

1. **🎯 Transfer Learning YOLO** : Adaptation depuis COCO vers surveillance
2. **📊 Dataset Personnalisé** : Classes spécialisées pour la surveillance
3. **⚡ Optimisation** : Export ONNX/TensorRT pour déploiement
4. **📈 Évaluation** : Métriques complètes et comparaisons
5. **🔧 Intégration** : Code prêt pour le système principal

### 📊 Résultats Attendus :

Le modèle fine-tuné devrait montrer :
- **🎯 Classes spécialisées** mieux adaptées à la surveillance
- **📈 Précision améliorée** pour objets de surveillance
- **⚡ Performance optimisée** pour déploiement temps réel
- **🔧 Intégration facile** dans le système existant

### 📊 Métriques Importantes :

À surveiller en production :
- **mAP@0.5** > 0.7 pour les classes critiques (person, handbag)
- **Précision** > 0.8 pour minimiser les faux positifs
- **Rappel** > 0.75 pour ne pas manquer les vrais positifs
- **Temps d'inférence** < 50ms pour temps réel

### 🚀 Prochaines Étapes Recommandées :

1. **📊 Dataset Réel** : Collectez et annotez vos données de surveillance
2. **🔄 Fine-tuning Itératif** : Améliorez avec données terrain
3. **📈 Validation Croisée** : Testez sur différents magasins/conditions
4. **⚡ Optimisation Edge** : TensorRT/OpenVINO pour déploiement embarqué
5. **📊 Monitoring Production** : Métriques temps réel et drift detection

### 💡 Conseils pour la Production :

#### Dataset
- **📸 Images variées** : Différentes conditions (éclairage, foule, heures)
- **🏷️ Annotations précises** : IoU > 0.8, validation par experts
- **⚖️ Classes équilibrées** : SMOTE ou oversampling pour classes rares
- **📈 Augmentation** : Rotation, échelle, luminosité selon contexte

#### Modèle
- **🎛️ Hyperparamètres** : Grid search sur lr, batch_size, epochs
- **🔍 Seuils optimaux** : Courbes PR pour balance précision/rappel
- **📊 Validation** : K-fold ou temporal split pour robustesse
- **⚡ Ensemble** : Combinaison de plusieurs modèles si ressources suffisantes

#### Déploiement
- **🖥️ Infrastructure** : GPU pour temps réel, fallback CPU
- **📈 Monitoring** : Latence, mémoire, drift de données
- **🔄 CI/CD** : Pipeline automatique retrain/deploy
- **👥 Feedback Loop** : Annotations terrain pour amélioration continue

### 🛠️ Intégration Système Complet :

```python
# Dans votre système principal
from surveillance_yolo_integration import SurveillanceYOLODetector

# Remplacer YOLODetector par SurveillanceYOLODetector
detector = SurveillanceYOLODetector(
    model_path="models/surveillance_yolo.pt",
    confidence_threshold=0.3
)

# Le reste du pipeline reste identique
results = detector.detect(frame)
```

---

**🎉 Félicitations ! Vous maîtrisez maintenant le transfer learning YOLO pour la surveillance.**

**📚 Ressources Complémentaires :**
- [Ultralytics YOLOv8 Docs](https://docs.ultralytics.com/)
- [Transfer Learning Best Practices](https://arxiv.org/abs/1411.1792)
- [YOLO Transfer Learning Guide](https://github.com/ultralytics/ultralytics)
- [Système de Surveillance Complet](https://github.com/elfried-kinzoun/intelligent-surveillance-system)

*Développé pour révolutionner la détection d'objets en surveillance*